**STEP-1 INSTALLING REQ PACKAGES AND LIB**

In [ ]:
!nvidia-smi

Sun Oct 26 09:23:07 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   55C    P0             27W /   70W |    7010MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# the required packages
!pip install pypdf
!pip install -q transformers accelerate bitsandbytes langchain
!pip install sentence_transformers
!pip install llama-index==0.9.39
!pip install langchain-huggingface
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 23.7 MB/s eta 0:00:00


In [ ]:
# the required lib
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms import HuggingFaceLLM
from llama_index.prompts.prompts import SimpleInputPrompt
from llama_index.text_splitter import TokenTextSplitter
from transformers import AutoTokenizer
import torch
from transformers import AutoModelForCausalLM
from transformers import BitsAndBytesConfig
from llama_index.embeddings.langchain import LangchainEmbedding
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

**STEP-2 UPLODING THE DATA**

In [ ]:
!mkdir Data

In [ ]:
documents = SimpleDirectoryReader(input_files=["/content/Data/Redmon_You_Only_Look_CVPR_2016_paper.pdf"]).load_data()

In [ ]:
documents

[Document(id_='efb90a7d-b55f-4316-9a88-19d1f0e5059d', embedding=None, metadata={'page_label': '1', 'file_name': 'Redmon_You_Only_Look_CVPR_2016_paper.pdf', 'file_path': '/content/Data/Redmon_You_Only_Look_CVPR_2016_paper.pdf', 'file_type': 'application/pdf', 'file_size': 1270136, 'creation_date': '2025-10-26', 'last_modified_date': '2025-10-26', 'last_accessed_date': '2025-10-26'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Y ou Only Look Once:\nUniﬁed, Real-Time Object Detection\nJoseph Redmon ∗, Santosh Divvala ∗†, Ross Girshick ¶ , Ali Farhadi ∗†\nUniversity of W ashington ∗ , Allen Institute for AI † , Facebook AI Research ¶\nhttp://pjreddie.com/yolo/\nAbstract\nW e present YOLO, a new approach to object detection.\nPrior work on object detectio

**STEP 3 - SETTING UP THE PROMPT TEMPLATE**

In [ ]:
system_prompt = """ you are a Q&A assitant. your goals is to answer questions as
accurately as possible based on the context provided. if the
context is not helpful, you can ask the user for more information and if informations
is not present simple reply the information you are looking for is not present."""
query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

**STEP 4 - LOADING AND INITIALIZING THE MODEL**

In [ ]:
!huggingface-cli login

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
The token `GEN_AI` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as

In [ ]:
model_name = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# for loading the model in 4 bit memory
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)
# for  conversion of text into tokens
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    torch_dtype=torch.float16
)
# for conversion of tokens into vectors for storing in vector db
llm = HuggingFaceLLM(
    model=model,
    tokenizer=tokenizer,
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

**STEP 5 - INITIALIZING THE HUGGING FACE EMBEDDINGS**

In [ ]:
embeddings = HuggingFaceEmbedding(model_name="sentence-transformers/all-mpnet-base-v2")

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

**STEP 6 - CREATING THE VECTORE STORE**

In [ ]:
service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embeddings
)

In [ ]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

**STEP 7 - GENERATING THE RESPONSE**

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("give summary of the document")
print(response)